In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import cProfile #for checking the nr of calls and execution time
import pstats
from pstats import SortKey

from tqdm.notebook import tqdm
import numpy as np
import multiple_planets_gas_acc as code_gas
import functions_pebble_accretion as peb
from functions import *
import functions_plotting as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as ub
import pandas as pd
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator, MultipleLocator, AutoMinorLocator
from matplotlib import cm, ticker
from matplotlib import colors
import matplotlib.gridspec as gridspec
import matplotlib.patches as patch
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

import matplotlib.lines as mlines 

color = mpl.colormaps["YlOrRd"].reversed()(np.linspace(0, 0.7, code_gas.sim_params.nr_planets))


In [ ]:
def kepler_3_law (period):
    return ((const.G.cgs*const.M_sun.cgs/(4*np.pi**2)*period**2)**(1/3)).to(u.au)
t = (400*u.day).to(u.s)
kepler_3_law(t)

In [ ]:
# disc parameters
params_dict = {'St_const': None, 
               'M_dot_star': None,
               'iceline_radius': None,
                'alpha': 1e-2,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'epsilon_el': 1e-2,
                'epsilon_heat':0.5,
                'v_frag': (1 * u.m/u.s).to(u.au/u.Myr).value,
                'H_r_model': "irradiated", 
                }
params = code_gas.Params(**params_dict)
print(params.iceline_flux_change)

t_fin = 5 #Myr, end of sim
N_steps = 100
# Number of samples to generate
num_samples = 150
import scipy.stats as stats

seed = 42
seed_t0 = 11
see_ap0 = 7

#inner embryo
t0_inner_samples = stats.uniform.rvs(loc=0.1, scale=0.9, size=num_samples, random_state=seed_t0)
print(t0_inner_samples)
a_p0_inner_samples = stats.loguniform.rvs(1e-1, 1e1, size=num_samples, random_state=see_ap0)
t0_inner = ([t0_inner_samples] * np.ones(len(a_p0_inner_samples))) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
# outer embryo
a_p0_outer = np.array([30])
t0_outer = ([1] * np.ones(len(a_p0_outer)) ) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
sigma_gas_outer = sigma_gas_steady_state(a_p0_outer, t0_outer, params)
m0_outer = M0_pla(a_p0_outer, t0_outer, sigma_gas_outer, params)

fig,axs = plt.subplots(1,1, figsize=(6,6))
axs.scatter(a_p0_inner_samples, t0_inner_samples, label='Inner planet', color='blue')
axs.set_xscale('log')
axs.set_xscale('log')
axs.set_xlabel('a [AU]')
axs.set_ylabel('t [Myr]')
axs.set_title('Inner planet initial positions and times')
axs.axhline(0.1,  color = 'grey')
axs.axhline(1,  color = 'grey')
axs.axvline(0.1,  color = 'grey')
axs.axvline(10,  color = 'grey')



In [ ]:

simulations = []
parameters = []
sim_parameters = []

for  a_p0_in, t0_in in tm(zip(a_p0_inner_samples, t0_inner_samples)):
    #initial conditions: both a_p0 and m0 take the outer planet and one of the inner planets
    sigma_gas_inner = sigma_gas_steady_state(a_p0_in, t0_in, params)
    m0_in = M0_pla(a_p0_in, t0_in, sigma_gas_inner, params)
 
    a_p0 = np.array([*a_p0_outer, a_p0_in])
    m_0 = np.array([*m0_outer, m0_in])
    t0 = np.array([*t0_outer, t0_in])
    print(a_p0, m_0, t0)

    sim_params_dict = {'N_step': N_steps,
                    'm0': m_0,
                    'a_p0': a_p0,
                    't0': t0,
                    't_fin': t_fin,
                }
    sim_params = code_gas.SimulationParams(**sim_params_dict)
    peb_acc = code_gas.PebbleAccretion(simplified_acc=False)
    gas_acc = peb.GasAccretion()

    result = code_gas.simulate_euler(migration = True, filtering = True, peb_acc = peb_acc, gas_acc=gas_acc, params=params, sim_params=sim_params)
    simulations.append(result)
    parameters.append(params)
    sim_parameters.append(sim_params)



In [ ]:
# disc parameters
params_dict = {'St_const': None, 
               'M_dot_star': None,
               'iceline_radius': None,
                'alpha': 1e-2,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'epsilon_el': 1e-2,
                'epsilon_heat':0.5,
                'v_frag': (1 * u.m/u.s).to(u.au/u.Myr).value,
                'H_r_model': "Lambrechts_mixed",

                }
params_alpha = code_gas.Params(**params_dict)
print(params_alpha.H_r_model)

# def lognuniform(low=0, high=1, size=None, base=10):
#     return np.power(base, np.random.uniform(low, high, size))
# Number of samples to generate
num_samples = 150
import scipy.stats as stats

seed = 42
seed_t0 = 11
see_ap0 = 7
t0_inner_samples = stats.uniform.rvs(0.1, 1, size=num_samples, random_state=seed_t0)

#t0_inner_samples = stats.loguniform.rvs(0.1, 1, size=num_samples, random_state=seed_t0)
#t0_inner_samples = ([2e5] * np.ones(len(a_p0_inner_samples)) * u.yr).to(u.s) 

a_p0_inner_samples = stats.loguniform.rvs(1e-1, 1e1, size=num_samples, random_state=see_ap0)
# # Print the generated pairs
t0_inner = ([t0_inner_samples] * np.ones(len(a_p0_inner_samples))) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
# outer embryo
a_p0_outer = np.array([30])
t0_outer = ([0.1] * np.ones(len(a_p0_outer)) ) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
sigma_gas_outer = sigma_gas_steady_state(a_p0_outer, t0_outer, params_alpha)
m0_outer = M0_pla(a_p0_outer, t0_outer, sigma_gas_outer, params_alpha)

simulations_ice = []
parameters_ice = []
sim_parameters_ice = []
t_fin = 3 #Myr, emnd of sim

for  a_p0_in, t0_in in tqdm(zip(a_p0_inner_samples, t0_inner_samples)):
    #initial conditions: both a_p0 and m0 take the outer planet and one of the inner planets
    sigma_gas_inner = sigma_gas_steady_state(a_p0_in, t0_in, params_alpha)
    m0_in = M0_pla(a_p0_in, t0_in, sigma_gas_inner, params_alpha)
 
    a_p0 = np.array([*a_p0_outer, a_p0_in])
    m_0 = np.array([*m0_outer, m0_in])
    t0 = np.array([*t0_outer, t0_in])
    print(a_p0, m_0, t0)

    sim_params_dict = {'N_step': 1000,
                    'm0': m_0,
                    'a_p0': a_p0,
                    't0': t0,
                    't_fin': t_fin,

                }
    sim_params = code_gas.SimulationParams(**sim_params_dict)
    peb_acc = code_gas.PebbleAccretion(simplified_acc=False)
    gas_acc = peb.GasAccretion()

    result = code_gas.simulate_euler(migration = True, filtering = True, peb_acc = peb_acc, gas_acc=gas_acc, params=params_alpha, sim_params=sim_params)
    simulations_ice.append(result)
    parameters_ice.append(params_alpha)
    sim_parameters_ice.append(sim_params)



In [ ]:
# disc parameters
params_dict = {'St_const': None, 
               'M_dot_star': None,
               'iceline_radius': None,
                'alpha': 1e-2,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'epsilon_el': 1,
                'epsilon_heat':1,
                'v_frag': (1 * u.m/u.s).to(u.au/u.Myr).value,
                'H_r_model': "Lambrechts_mixed",

                }
params_alpha = code_gas.Params(**params_dict)
print(params_alpha.H_r_model)

# def lognuniform(low=0, high=1, size=None, base=10):
#     return np.power(base, np.random.uniform(low, high, size))
# Number of samples to generate
num_samples = 150
import scipy.stats as stats

seed = 42
seed_t0 = 11
see_ap0 = 7
t0_inner_samples = stats.uniform.rvs(0.1, 1, size=num_samples, random_state=seed_t0)

#t0_inner_samples = stats.loguniform.rvs(0.1, 1, size=num_samples, random_state=seed_t0)
#t0_inner_samples = ([2e5] * np.ones(len(a_p0_inner_samples)) * u.yr).to(u.s) 

a_p0_inner_samples = stats.loguniform.rvs(1e-1, 1e1, size=num_samples, random_state=see_ap0)
# # Print the generated pairs
t0_inner = ([t0_inner_samples] * np.ones(len(a_p0_inner_samples))) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
# outer embryo
a_p0_outer = np.array([30])
t0_outer = ([0.2] * np.ones(len(a_p0_outer)) ) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
sigma_gas_outer = sigma_gas_steady_state(a_p0_outer, t0_outer, params_alpha)
m0_outer = M0_pla(a_p0_outer, t0_outer, sigma_gas_outer, params_alpha)

simulations_alpha = []
parameters_alpha = []
sim_parameters_alpha = []
t_fin = 3 #Myr, emnd of sim

for  a_p0_in, t0_in in tqdm(zip(a_p0_inner_samples, t0_inner_samples)):
    #initial conditions: both a_p0 and m0 take the outer planet and one of the inner planets
    sigma_gas_inner = sigma_gas_steady_state(a_p0_in, t0_in, params_alpha)
    m0_in = M0_pla(a_p0_in, t0_in, sigma_gas_inner, params_alpha)
 
    a_p0 = np.array([*a_p0_outer, a_p0_in])
    m_0 = np.array([*m0_outer, m0_in])
    t0 = np.array([*t0_outer, t0_in])
    print(a_p0, m_0, t0)

    sim_params_dict = {'N_step': 1000,
                    'm0': m_0,
                    'a_p0': a_p0,
                    't0': t0,
                    't_fin': t_fin,

                }
    sim_params = code_gas.SimulationParams(**sim_params_dict)
    peb_acc = code_gas.PebbleAccretion(simplified_acc=False)
    gas_acc = peb.GasAccretion()

    result = code_gas.simulate_euler(migration = True, filtering = True, peb_acc = peb_acc, gas_acc=gas_acc, params=params_alpha, sim_params=sim_params)
    simulations_alpha.append(result)
    parameters_alpha.append(params_alpha)
    sim_parameters_alpha.append(sim_params)



In [ ]:

fig, axs = plt.subplots(1,3, figsize=(18, 6))
title = ["Irradiation", "Surface heating", "Midplane heating"]
a_p0 = np.geomspace(5e-3, 1e2, num = 1000)

alpha_transp = 0.2

############### DISCS loops ########################
for k, (simulations, parameters, sim_parameters) in enumerate(zip([simulations], [parameters], [sim_parameters])):

#for k, (simulations, parameters, sim_parameters) in enumerate(zip([simulations, simulations_ice, simulations_alpha], [parameters, parameters_ice, parameters_alpha], [sim_parameters, sim_parameters_ice, sim_parameters_alpha])):
    for i in range(len(simulations)):
        sim = simulations[i]
        params = parameters[i]
        sim_params = sim_parameters[i]
        print(sim.time)
        for p in range(1,sim_params.nr_planets):
            idx = plot.idxs (axs, sim.time[p], sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                            sim.dR_dt[p], sim.dM_dt[p], params, True)
            iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
            stop_mig_idx = idx['stop_mig_idx'].values[0]
            cmap =  mpl.cm.inferno.reversed()
            norm = mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin)
            colors = cmap(norm(sim_params.t0[p]))
            axs[k].scatter(sim.position[p,0].to(u.au), sim.mass[p,0].to(u.M_earth), facecolors='none',edgecolors = colors, norm=norm, 
            cmap = cmap)

            if iso_idx != 0:
                axs[k].scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.M_earth),
                                color =  cmap(norm(sim.time[iso_idx].to(u.Myr).value)), facecolors='none', marker = 'v')

            axs[k].scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.M_earth), 
                        color =cmap(norm(sim.time[stop_mig_idx].to(u.Myr).value)))
            
    #print(k, planet_counter(simulations, parameters, sim_parameters))
    for i in range(1):
        for p in [0]:
            print(sim.mass[p])
            idx = plot.idxs (axs, sim.time[p], sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                            sim.dR_dt[p], sim.dM_dt[p], params, True)
            iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
            stop_mig_idx = idx['stop_mig_idx'].values[0]
            dt = (sim.time[:iso_idx]).to(u.Myr)
            sc = axs[k].scatter(sim.position[p,:iso_idx].to(u.au), sim.mass[p,:iso_idx].to(u.M_earth), c = dt, 
                                norm=mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin), 
            cmap = mpl.cm.inferno.reversed())
            axs[k].loglog(sim.position.to(u.au)[p,iso_idx:stop_mig_idx], sim.mass[p,iso_idx:stop_mig_idx].to(u.earthMass), color='grey', linewidth=6, 
                        alpha = alpha_transp)
            axs[k].scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.earthMass), marker = 'x', color = 'grey', s = 100, zorder=100)
            axs[k].scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.earthMass), 
                        color = cmap(norm(sim.time[iso_idx].to(u.Myr).value)), marker = 'v', s = 200, zorder=100)
        axs[k].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_in, params), color = "slateblue", linestyle =':', zorder = 0)
        axs[k].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params), color = "slateblue", linestyle =':', zorder = 0)
        axs[k].fill_between(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params), M_peb_iso(a_p0, sim_params.t_in, params),  color='slateblue', alpha=0.1)    #plot the magnetic cavity and shade the region inside it (from initial to final position)


cbar_ax = fig.add_axes([0.95, 0.15, 0.03, 0.7])
cbar = fig.colorbar(sc, cax=cbar_ax)	
# Manually set the colorbar boundaries and ticks
cbar_ax.yaxis.set_major_locator(LogLocator(base=10.0, subs=[1.0, 5.0]))
cbar_ax.yaxis.set_major_formatter(LogFormatter())
cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(plot.custom_log_formatter))
cbar.set_label('accretion timescale [Myr]', fontsize=20, labelpad=15)
cbar.ax.tick_params(axis = 'both', which = 'major', size = 13, labelsize = 13)
cbar.ax.tick_params(axis = 'both', which = 'minor', size = 9)
#plot the initial mass line
from matplotlib import pyplot as plt, ticker as mticker

for j in range(3):
    # #SE
    axs[j].fill_betweenx([1, 20], 0.001, 1, color='brown', alpha=0.1)
    #terrestrial embryo
    axs[j].fill_betweenx([0.01, 1], 0.1, 10, color='black', alpha=0.1)

    # gas giants
    axs[j].fill_betweenx([1e2, 6e3], 0.001, 0.1, color='aquamarine', alpha=0.1)
    axs[j].fill_betweenx([1e2, 6e3], 0.1, 2, color='gold', alpha=0.1)
    axs[j].fill_betweenx([1e2, 6e3], 2, 10, color='purple', alpha=0.1)


    plot.HD219134(axs[j], color = 'red')
    plot.solar_system(axs[j], color = 'green')
    axs[j].axvline(r_magnetic_cavity(sim_params.t_in, params), linestyle = '-.', color = 'grey', alpha = 0.1)
    axs[j].axvline(r_magnetic_cavity(sim_params.t_fin, params), linestyle = '-.', color = 'grey', alpha = 0.1)
    axs[j].axvspan(r_magnetic_cavity(sim_params.t_in, params), r_magnetic_cavity(sim_params.t_fin, params),facecolor='none', hatch='/', edgecolor='gray', alpha =alpha_transp)

    axs[j].set_xlabel('r [AU]', fontsize = 20, labelpad=20)
    axs[j].set_title(title[j], fontsize = 20)
    axs[j].set_xlim(5e-3, 1e2)
    axs[j].tick_params(axis = "both", which = "major", direction = 'in', size = 15)
    axs[j].tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
    axs[j].set_ylim(5e-8, 1e3)

    axs[j].yaxis.set_major_locator(mticker.LogLocator(numticks=999))
    axs[j].yaxis.set_minor_locator(mticker.LogLocator(numticks=999, subs="auto"))

axs[0].set_ylabel('M [$M_{\oplus}$]', fontsize = 20, labelpad=20)
plt.savefig("figures/pop_synth/pop_synt_gas_models_longer_t", bbox_inches='tight')

In [ ]:
(0.001*u.Myr).to(u.kyr)

In [ ]:

fig, axs = plt.subplots(1,3, figsize=(18, 6))
title = ["Irradiation", "Surface heating", "Midplane heating"]
a_p0 = np.geomspace(5e-3, 1e2, num = 1000)

alpha_transp = 0.2
"""
for i in range(len(simulations)):
    sim = simulations[i]
    params = parameters[i]
    sim_params = sim_parameters[i]
    for p in range(1,sim_params.nr_planets):
        idx = plot.idxs (axs, sim.time[p], sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                        sim.dR_dt[p], sim.dM_dt[p], params, True)
        iso_idx = idx['isolation_idx'].values[0]
        stop_mig_idx = idx['stop_mig_idx'].values[0]

        cmap =  mpl.cm.inferno.reversed()
        norm = mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin)
        colors = cmap(norm(sim_params.t0[p]))
        axs[0].scatter(sim.position[p,0].to(u.au), sim.mass[p,0].to(u.M_earth), facecolors='none',edgecolors = colors, norm=norm, 
        cmap = cmap)

        if iso_idx<sim.mass[p].size:
            #axs[0].scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.M_earth), color = 'slateblue', facecolors='none')
            axs[0].scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.M_earth), 
                           color =  cmap(norm(sim.time[iso_idx].to(u.Myr).value)), facecolors='none', marker = 'v')

        axs[0].scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.M_earth), 
                       color = cmap(norm(sim.time[stop_mig_idx].to(u.Myr).value)))
        
        # # #to compare
        # axs[1].scatter(sim.position[p,-1].to(u.au), sim.mass[p,-1].to(u.M_earth), color = 'red', zorder = 0)
        # axs[2].scatter(sim.position[p,-1].to(u.au), sim.mass[p,-1].to(u.M_earth), color = 'red', zorder = 0)

print(planet_counter(simulations, parameters, sim_parameters))
for i in range(1):
    for p in [0]:
        idx = plot.idxs (axs, sim.time[p], sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                        sim.dR_dt[p], sim.dM_dt[p], params, True)
        iso_idx = idx['isolation_idx'].values[0]
        stop_mig_idx = idx['stop_mig_idx'].values[0]
        dt = (sim.time[:iso_idx]).to(u.Myr)
        sc = axs[0].scatter(sim.position[p,:iso_idx].to(u.au), sim.mass[p,:iso_idx].to(u.M_earth), c = dt, 
                            norm=mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin), 
        cmap = mpl.cm.inferno.reversed())
        axs[0].loglog(sim.position.to(u.au)[p,iso_idx:stop_mig_idx], sim.mass[p,iso_idx:stop_mig_idx].to(u.earthMass), 
                      color='grey', linewidth=6, alpha = alpha_transp)
        axs[0].scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.earthMass), marker = 'x', color = 'grey', s = 100, zorder=100)
        axs[0].scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.earthMass), 
                       color = cmap(norm(sim.time[iso_idx].to(u.Myr).value)), marker = 'v', s = 200, zorder=100)
        #plot the isolation mass line
        axs[0].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_in, params), color = "slateblue", linestyle =':', zorder = 0)
        axs[0].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params), color = "slateblue", linestyle =':', zorder = 0)
        axs[0].fill_between(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params), M_peb_iso(a_p0, sim_params.t_in, params),  color='slateblue', alpha=0.1)    #plot the magnetic cavity and shade the region inside it (from initial to final position)
# axs[0].axvline(iceline(sim_params.t_fin, 170, params), color= 'lightblue', linestyle = '-.')
# axs[0].axvline(iceline(sim_params.t_in, 170, params), color= 'lightblue', linestyle = '-.')
# axs[0].axvspan(iceline(sim_params.t_fin, 170, params), iceline(sim_params.t_in, 170, params),  color='lightblue', alpha=0.1)    #plot the magnetic cavity and shade the region inside it (from initial to final position)
"""

################# SURFACE HEATING ############################


for i in range(len(simulations_ice)):
    sim_ice = simulations_ice[i]
    params_ice = parameters_ice[i]
    sim_params_ice = sim_parameters_ice[i]
    for p in range(1,sim_params_ice.nr_planets):
        idx = plot.idxs (axs, sim_ice.time[p], sim_ice.mass[p], sim_ice.position[p], sim_ice.filter_fraction[p], 
                        sim_ice.dR_dt[p], sim_ice.dM_dt[p], params_ice, True)
        
        iso_idx = np.argmax(sim_ice.mass[p] > M_peb_iso(sim_ice.position[p].value, sim_ice.time.value, params_ice)*u.M_earth)
        stop_mig_idx = idx['stop_mig_idx'].values[0]

        cmap =  mpl.cm.inferno.reversed()
        norm = mpl.colors.LogNorm(vmin = sim_params_ice.t_in, vmax = sim_params_ice.t_fin)
        colors = cmap(norm(sim_params_ice.t0[p]))
        axs[1].scatter(sim_ice.position[p,0].to(u.au), sim_ice.mass[p,0].to(u.M_earth), facecolors='none',edgecolors = colors, norm=norm, 
        cmap = cmap)

        if iso_idx != 0:
            axs[1].scatter(sim_ice.position[p,iso_idx].to(u.au), sim_ice.mass[p,iso_idx].to(u.M_earth),
                            color =  cmap(norm(sim_ice.time[iso_idx].to(u.Myr).value)), facecolors='none', marker = 'v')

        axs[1].scatter(sim_ice.position[p,stop_mig_idx].to(u.au), sim_ice.mass[p,stop_mig_idx].to(u.M_earth), 
                       color =cmap(norm(sim_ice.time[stop_mig_idx].to(u.Myr).value)))
        # # #to compare
        # axs[0].scatter(sim_alpha.position[p,-1].to(u.au), sim_alpha.mass[p,-1].to(u.M_earth), color = 'purple', zorder = 0)
        # axs[1].scatter(sim_alpha.position[p,-1].to(u.au), sim_alpha.mass[p,-1].to(u.M_earth), color = 'purple', zorder = 0)
print(planet_counter(simulations_ice, parameters_ice, sim_parameters_ice))
for i in range(1):
    for p in [0]:
        idx = plot.idxs (axs, sim_ice.time[p], sim_ice.mass[p], sim_ice.position[p], sim_ice.filter_fraction[p], 
                        sim_ice.dR_dt[p], sim_ice.dM_dt[p], params_ice, True)
        iso_idx = np.argmax(sim_ice.mass[p] > M_peb_iso(sim_ice.position[p].value, sim_ice.time.value, params_ice)*u.M_earth)
        stop_mig_idx = idx['stop_mig_idx'].values[0]
        dt = (sim_ice.time[:iso_idx]).to(u.Myr)
        sc = axs[1].scatter(sim_ice.position[p,:iso_idx].to(u.au), sim_ice.mass[p,:iso_idx].to(u.M_earth), c = dt, 
                            norm=mpl.colors.LogNorm(vmin = sim_params_ice.t_in, vmax = sim_params_ice.t_fin), 
        cmap = mpl.cm.inferno.reversed())
        axs[1].loglog(sim_ice.position.to(u.au)[p,iso_idx:stop_mig_idx], sim_ice.mass[p,iso_idx:stop_mig_idx].to(u.earthMass), color='grey', linewidth=6, 
                      alpha = alpha_transp)
        axs[1].scatter(sim_ice.position[p,stop_mig_idx].to(u.au), sim_ice.mass[p,stop_mig_idx].to(u.earthMass), marker = 'x', color = 'grey', s = 100, zorder=100)
        axs[1].scatter(sim_ice.position[p,iso_idx].to(u.au), sim_ice.mass[p,iso_idx].to(u.earthMass), 
                       color = cmap(norm(sim_ice.time[iso_idx].to(u.Myr).value)), marker = 'v', s = 200, zorder=100)
    axs[1].loglog(a_p0, M_peb_iso(a_p0, sim_params_ice.t_in, params_ice), color = "slateblue", linestyle =':', zorder = 0)
    axs[1].loglog(a_p0, M_peb_iso(a_p0, sim_params_ice.t_fin, params_ice), color = "slateblue", linestyle =':', zorder = 0)
    axs[1].fill_between(a_p0, M_peb_iso(a_p0, sim_params_ice.t_fin, params_ice), M_peb_iso(a_p0, sim_params_ice.t_in, params_ice),  color='slateblue', alpha=0.1)    #plot the magnetic cavity and shade the region inside it (from initial to final position)


############### MIDPLANE HEATING ########################

for i in range(len(simulations_alpha)):
    sim_alpha = simulations_alpha[i]
    params_alpha = parameters_alpha[i]
    sim_params_alpha = sim_parameters_alpha[i]
    for p in range(1,sim_params_alpha.nr_planets):
        idx = plot.idxs (axs, sim_alpha.time[p], sim_alpha.mass[p], sim_alpha.position[p], sim_alpha.filter_fraction[p], 
                        sim_alpha.dR_dt[p], sim_alpha.dM_dt[p], params_alpha, True)
        iso_idx = np.argmax(sim_alpha.mass[p] > M_peb_iso(sim_alpha.position[p].value, sim_alpha.time.value, params_alpha)*u.M_earth)
        stop_mig_idx = idx['stop_mig_idx'].values[0]
        cmap =  mpl.cm.inferno.reversed()
        norm = mpl.colors.LogNorm(vmin = sim_params_alpha.t_in, vmax = sim_params_alpha.t_fin)
        colors = cmap(norm(sim_params_alpha.t0[p]))
        axs[2].scatter(sim_alpha.position[p,0].to(u.au), sim_alpha.mass[p,0].to(u.M_earth), facecolors='none',edgecolors = colors, norm=norm, 
        cmap = cmap)

        if iso_idx<sim_alpha.mass[p].size:
            #axs[2].scatter(sim_alpha.position[p,iso_idx].to(u.au), sim_alpha.mass[p,iso_idx].to(u.M_earth), color = 'black')
            axs[2].scatter(sim_alpha.position[p,iso_idx].to(u.au), sim_alpha.mass[p,iso_idx].to(u.M_earth),
                            color =  cmap(norm(sim_alpha.time[iso_idx].to(u.Myr).value)), facecolors='none', marker = 'v')

        axs[2].scatter(sim_alpha.position[p,stop_mig_idx].to(u.au), sim_alpha.mass[p,stop_mig_idx].to(u.M_earth), 
                       color =cmap(norm(sim_alpha.time[stop_mig_idx].to(u.Myr).value)))
        # # #to compare
        # axs[0].scatter(sim_alpha.position[p,-1].to(u.au), sim_alpha.mass[p,-1].to(u.M_earth), color = 'purple', zorder = 0)
        # axs[1].scatter(sim_alpha.position[p,-1].to(u.au), sim_alpha.mass[p,-1].to(u.M_earth), color = 'purple', zorder = 0)
print(planet_counter(simulations_alpha, parameters_alpha, sim_parameters_alpha))
for i in range(1):
    for p in [0]:
        idx = plot.idxs (axs, sim_alpha.time[p], sim_alpha.mass[p], sim_alpha.position[p], sim_alpha.filter_fraction[p], 
                        sim_alpha.dR_dt[p], sim_alpha.dM_dt[p], params_alpha, True)
        iso_idx = np.argmax(sim_alpha.mass[p] > M_peb_iso(sim_alpha.position[p].value, sim_alpha.time.value, params_alpha)*u.M_earth)
        stop_mig_idx = idx['stop_mig_idx'].values[0]
        dt = (sim_alpha.time[:iso_idx]).to(u.Myr)
        sc = axs[2].scatter(sim_alpha.position[p,:iso_idx].to(u.au), sim_alpha.mass[p,:iso_idx].to(u.M_earth), c = dt, 
                            norm=mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin), 
        cmap = mpl.cm.inferno.reversed())
        axs[2].loglog(sim_alpha.position.to(u.au)[p,iso_idx:stop_mig_idx], sim_alpha.mass[p,iso_idx:stop_mig_idx].to(u.earthMass), color='grey', linewidth=6, 
                      alpha = alpha_transp)
        axs[2].scatter(sim_alpha.position[p,stop_mig_idx].to(u.au), sim_alpha.mass[p,stop_mig_idx].to(u.earthMass), marker = 'x', color = 'grey', s = 100, zorder=100)
        axs[2].scatter(sim_alpha.position[p,iso_idx].to(u.au), sim_alpha.mass[p,iso_idx].to(u.earthMass), 
                       color = cmap(norm(sim_alpha.time[iso_idx].to(u.Myr).value)), marker = 'v', s = 200, zorder=100)
    axs[2].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_in, params_alpha), color = "slateblue", linestyle =':', zorder = 0)
    axs[2].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params_alpha), color = "slateblue", linestyle =':', zorder = 0)
    axs[2].fill_between(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params_alpha), M_peb_iso(a_p0, sim_params.t_in, params_alpha),  color='slateblue', alpha=0.1)    #plot the magnetic cavity and shade the region inside it (from initial to final position)

# axs[2].axvline(iceline(sim_params_alpha.t_fin, 170, params_alpha), color= 'lightblue', linestyle = '-.')
# axs[2].axvline(iceline(sim_params_alpha.t_in, 170, params_alpha), color= 'lightblue', linestyle = '-.')
# axs[2].axvspan(iceline(sim_params_alpha.t_fin, 170, params_alpha), iceline(sim_params_alpha.t_in, 170, params_alpha),  color='lightblue', alpha=0.1)    #plot the magnetic cavity and shade the region inside it (from initial to final position)




cbar_ax = fig.add_axes([0.95, 0.15, 0.03, 0.7])
cbar = fig.colorbar(sc, cax=cbar_ax)	
# Manually set the colorbar boundaries and ticks
cbar_ax.yaxis.set_major_locator(LogLocator(base=10.0, subs=[1.0, 5.0]))
cbar_ax.yaxis.set_major_formatter(LogFormatter())
cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(plot.custom_log_formatter))
cbar.set_label('accretion timescale [Myr]', fontsize=20, labelpad=15)
cbar.ax.tick_params(axis = 'both', which = 'major', size = 13, labelsize = 13)
cbar.ax.tick_params(axis = 'both', which = 'minor', size = 9)
#plot the initial mass line

# Extract the initial masses from each simulation
#initial_masses = [sim.position[:,0] for sim in sims[:][0]]
# Find the minimum initial mass
#min_initial_mass = min(initial_masses)
#print(min_initial_mass)
from matplotlib import pyplot as plt, ticker as mticker

for j in range(3):
    # #SE
    axs[j].fill_betweenx([1, 20], 0.001, 1, color='brown', alpha=0.1)
    #terrestrial embryo
    axs[j].fill_betweenx([0.01, 1], 0.1, 10, color='black', alpha=0.1)

    # gas giants
    axs[j].fill_betweenx([1e2, 6e3], 0.001, 0.1, color='aquamarine', alpha=0.1)
    axs[j].fill_betweenx([1e2, 6e3], 0.1, 2, color='gold', alpha=0.1)
    axs[j].fill_betweenx([1e2, 6e3], 2, 10, color='purple', alpha=0.1)


    plot.HD219134(axs[j], color = 'red')
    plot.solar_system(axs[j], color = 'green')
    axs[j].axvline(r_magnetic_cavity(sim_params.t_in, params), linestyle = '-.', color = 'grey', alpha = 0.1)
    axs[j].axvline(r_magnetic_cavity(sim_params.t_fin, params), linestyle = '-.', color = 'grey', alpha = 0.1)
    axs[j].axvspan(r_magnetic_cavity(sim_params.t_in, params), r_magnetic_cavity(sim_params.t_fin, params),facecolor='none', hatch='/', edgecolor='gray', alpha =alpha_transp)

    axs[j].set_xlabel('r [AU]', fontsize = 20, labelpad=20)
    axs[j].set_title(title[j], fontsize = 20)
    axs[j].set_xlim(5e-3, 1e2)
    axs[j].tick_params(axis = "both", which = "major", direction = 'in', size = 15)
    axs[j].tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
    axs[j].set_ylim(5e-8, 1e3)

    axs[j].yaxis.set_major_locator(mticker.LogLocator(numticks=999))
    axs[j].yaxis.set_minor_locator(mticker.LogLocator(numticks=999, subs="auto"))

axs[0].set_ylabel('M [$M_{\oplus}$]', fontsize = 20, labelpad=20)
plt.savefig("figures/pop_synth/pop_synt_gas_models_tuniform", bbox_inches='tight')